# Silcton MRI Processing and Analyses

Data, info on Virtual Silcton, analysis plan can all be found here: https://osf.io/ea99d/

In [1]:
#import packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from cycler import cycler
import seaborn as sns
# For statistics. Requires statsmodels 5.0 or more
from statsmodels.formula.api import ols



In [2]:
def zscore(column, data):
    #Calculate and return the z-score value of a variable from a DataFrame
    zscoredVariable = (data[column] - data[column].mean())/data[column].std(ddof=0)
    return zscoredVariable

In [3]:
def doAnova(dependentVar,data):
    data = data[np.isfinite(data[dependentVar])]
    grps = pd.unique(data.groups.values)
    d_data = {grp:data[dependentVar][data.groups == grp] for grp in grps}
    F, p = scipy.stats.f_oneway(d_data[1], d_data[2], d_data[3])
    print('F = ',F,'p = ',p)

In [4]:
def stderror(dependentVar):
    std = np.std(dependentVar)
    n = len(dependentVar)
    sem = std / np.sqrt(n)
    
    return sem

In [5]:
def saveTheFig(name1,name2,f):
    
    figName = 'scatter'+name1.capitalize()+name2.capitalize()+'.pdf'
    
    prompt = 'save figure with ' + name1.capitalize() + name2.capitalize() + '? y/n '
    
    a = input(prompt)

    if a.lower() == 'y':
        f.savefig(figName, format='pdf',transparent=True,)


In [6]:
def formatPlot(x,y,plot,linecolor,ax1):
    ax1.set_xlabel(x.name,fontsize=20)
    ax1.set_ylabel(y.name,fontsize=20)
    ax1.tick_params(axis='both', which='major', labelsize=14)
    fit = np.polyfit(x, y, deg=1) # calculate regression line
    ax1.plot(x, fit[0] * x + fit[1], color=linecolor,label='',linewidth=2) # print lines as black
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    pearsonR = 'r = ' + "{0:.2f}".format(r_value)
    locationOffset = (min(y) - max(y)) / 10
    location = (max(x)*fit[0] + fit[1]) + locationOffset
    ax1.text(max(x),location,pearsonR,horizontalalignment='right',fontsize=14,color=linecolor)
    ax1.legend(edgecolor='black',fontsize=14)
    fig = plt.gcf()
    fig.set_size_inches(10.5, 10.5)



In [7]:
def makeNavGroupPlot(xname,yname,d,savefig=False):

    """Will make a formatted scatter plot colored by all three groups"""
    
    x = normedData.loc[:,xname]
    y = normedData.loc[:,yname]
    f, ax1 = plt.subplots(1)
    group_dict = {1:['#00B050','Integrators'],2:['#E46C0A','Non-Integrators'],3:['#4BACC6','Imprecise Navigators']}



    for kind in group_dict:
        d = normedData[normedData.groups==kind]
        plt.scatter(d.loc[:,xname], d.loc[:,yname],
            c = group_dict[kind][0],label=group_dict[kind][1],s=60)
        plt.hlines(np.mean(d.loc[:,yname]), np.mean(d.loc[:,xname]) - stderror(d.loc[:,xname]), np.mean(d.loc[:,xname]) + stderror(d.loc[:,xname]), linestyle='dashed')
        plt.vlines(np.mean(d.loc[:,xname]), np.mean(d.loc[:,yname]) - stderror(d.loc[:,yname]), np.mean(d.loc[:,yname]) + stderror(d.loc[:,yname]), linestyle='dashed')
        plt.plot(np.mean(d.loc[:,xname]), np.mean(d.loc[:,yname]), marker='o', markersize=20
                 , color=group_dict[kind][0], markeredgecolor='black')
        formatPlot(d.loc[:,xname],d.loc[:,yname],f,group_dict[kind][0],ax1)

    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    pearsonR = 'r = ' + "{0:.2f}".format(r_value)
    
    fit = np.polyfit(x, y, deg=1) # calculate regression line
    ax1.plot(x, fit[0] * x + fit[1], color='gray',label='Total',linewidth=2) # print lines as black

    locationOffset = (max(y) - min(y)) / 20
    location = (max(x)*fit[0] + fit[1]) + locationOffset
    ax1.text(max(x),location,pearsonR,horizontalalignment='right', fontsize=14)

    if savefig:
        saveTheFig(x.name,y.name,f)



## Import and process data

In [39]:
#import and process data - including Z-scoring all columns

data = pd.read_csv('https://raw.githubusercontent.com/smweis/Silcton_MRI/master/public_data/DataAnalysisWith90Participants_Jupyter.csv')
#CHANGE ME TO 90 WHEN THE REST OF THE DATA IS AVAILABLE
data = data.iloc[0:86,:]


#variables
#data = pd.DataFrame()

#exclude outliers from total hippocampal volume (both high and low outliers)

#data = data[data['ASHS_right_total_hippo'] >  data['ASHS_right_total_hippo'].mean() - data['ASHS_right_total_hippo'].std()*3]
#data = data[data['ASHS_right_total_hippo'] <  data['ASHS_right_total_hippo'].mean() + data['ASHS_right_total_hippo'].std()*3]


#Normalize all the variables

for i in data:
    if i in ['Batch','ID','Education']:
        continue
    elif data[i].dtype != object:
        data[i] = zscore(i,data)
    else:    
        continue
        



#reverse the pointing ones to be higher = good score
data['pointBetween'] = -1*(data['pointBetween'])
data['pointWithin'] = -1*(data['pointWithin'])
data['pointTotal'] = -1*(data['pointTotal'])


#check any histograms
data['lhc_bodytail'].hist()
#normedData['ashs_rhc'].hist()



data.head()

KeyError: 'pointBetween'

In [22]:
normedData['gender'].value_counts()
data.Race.value_counts()

Female    52
Male      34
Name: gender, dtype: int64

,ID,Batch,Age,Education,English_First_Lang,Right_Handed,Gender,Race,WRAT,MRT,...,ASHS_left_ColSul,ASHS_left_ERC,ASHS_left_Meninges_PHC,ASHS_left_MISC,ASHS_left_OTSul,ASHS_left_PHC,ASHS_left_Posterior_hippocampus,ASHS_right_total_hippo,ASHS_left_total_hippo,ASHS_total_hippo
0,1002,1,0.953238,22,English,Right,Male,White,NaN,0.992920,...,-0.693648,-0.987027,0.814841,-0.271268,-1.001525,-1.312915,-0.416783,-0.852618,-0.588909,-0.739635
1,1003,1,-0.817902,15,English,Right,Female,Black,-1.215280,-1.783123,...,0.368212,-0.159557,-0.419625,-0.535286,-0.822559,1.433280,-1.261042,-0.214338,-0.131727,-0.177848
2,1004,1,-1.070922,15,English,Right,Male,Asian,0.730811,-0.006456,...,-1.008342,-0.875554,0.064373,-0.119598,-0.174871,0.533319,-1.712651,-1.258545,-1.291725,-1.301131
3,1005,1,-1.323942,13,English,Right,Female,Asian,1.055159,1.659170,...,0.760199,-0.783375,0.151384,0.554490,-0.538485,0.058931,-1.170249,-0.029675,-0.243966,-0.136493
4,1006,1,0.194178,14,English,Right,Male,Black,-0.566583,-1.116873,...,2.050259,1.028056,-0.561017,-0.293738,-0.052719,0.952633,0.013601,-0.585328,-0.388525,-0.499951


In [ ]:
makeNavGroupPlot('lhc_head','pointTotal',normedData)


## Create scatterplots

In [ ]:
# Figures for iNav Poster

makeNavGroupPlot('ashs_rhc','pointTotal',normedData)


In [ ]:
makeNavGroupPlot('ashs_post_rhc','pointTotal',normedData)


In [ ]:
makeNavGroupPlot('cortexVol','pointTotal',normedData)


In [ ]:
makeNavGroupPlot('rcaud','pointTotal',normedData)


In [ ]:
makeNavGroupPlot('ashs_rhc','sbsod',normedData)


In [ ]:
makeNavGroupPlot('ashs_rhc','mrt',normedData)


In [ ]:
makeNavGroupPlot('ashs_ant_rhc','pointTotal',normedData)

In [ ]:
# ANOVAS on three groups (not on poster)

doAnova('ashs_post_rhc',normedData)



In [ ]:
# additional analyses for control (numbers not on poster)

nonannormedData = normedData.dropna()


model_fit = ols("pointTotal ~ lhc_bodytail * gender * age * cortexVol", normedData).fit()
print(model_fit.summary())



In [ ]:
# fitted values (need a constant term for intercept)
model_fitted_y = model_fit.fittedvalues
# model residuals
model_residuals = model_fit.resid
# normalized residuals
model_norm_residuals = model_fit.get_influence().resid_studentized_internal
# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
# absolute residuals
model_abs_resid = np.abs(model_residuals)
# leverage, from statsmodels internals
model_leverage = model_fit.get_influence().hat_matrix_diag
# cook's distance, from statsmodels internals
model_cooks = model_fit.get_influence().cooks_distance[0]


plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)
plot_lm_1.axes[0] = sns.residplot(model_fitted_y, 'pointTotal', data=normedData, 
                          lowess=True, 
                          scatter_kws={'alpha': 0.5}, 
                          line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})
plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')


plt.show()



In [ ]:
def plot_corr(df,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''
    fig = plt.figure(figsize=(size,size))
    ax1 = fig.add_subplot(111)
    cmap = plt.cm.get_cmap('jet')
    a = df.corr()
    np.fill_diagonal(a.values,np.nan)
    masked_array = np.ma.array (a, mask=np.isnan(a))
    cmap.set_bad('black',1.)
    ax1.imshow(masked_array, interpolation='nearest', cmap=cmap)
    cax = ax1.imshow(a, interpolation="nearest", cmap=cmap)
    plt.xticks(range(len(df.columns)), df.columns);
    plt.yticks(range(len(df.columns)), df.columns);
    print(a)
    fig.colorbar(cax)
    return a

order = [0,19,21,10,11,9,12,13,14,]
cols = noOutliers.columns.tolist()

mylist = [ cols[i] for i in order]

orderedNormedData = noOutliers[mylist]
corrMat = plot_corr(orderedNormedData,20)
plt.show()

#a = noOutliers.corr()
#display(a.style.background_gradient())
nonanOrderedData = orderedNormedData.dropna()
g = sns.clustermap(nonanOrderedData)

"""Index(['brainVol', 'cortexVol', 'lhc', 'rhc', 'lamyg', 'ramyg', 'csf',
       'pointBetween', 'pointWithin', 'pointTotal', 'rcaud', 'lcaud', 'sbsod',
       'wrat', 'mrt', 'groups', 'age', 'batch', 'gender', 'ashs_rhc',
       'ashs_lhc', 'ashs_post_rhc', 'ashs_post_lhc', 'ashs_ant_rhc',
       'ashs_ant_lhc', 'ashs_total_hipp', 'ashs_rerc', 'ashs_lerc',
       'ashs_lphc', 'ashs_rphc', 'ashs_lbr35', 'ashs_rbr35', 'ashs_lbr36',
       'ashs_rbr36'],"""

In [ ]:
pwd